In [ ]:
!pip install transformers datasets pandas

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
import pandas as pd

df = pd.read_csv("/content/bitext-retail-banking-llm-chatbot-training-dataset.csv")

# İlk birkaç satırı inceleyelim
df.head()


In [ ]:
dataset = df[['instruction', 'response']]

In [ ]:
dataset = dataset.dropna()

In [ ]:
from sklearn.model_selection import train_test_split

train_dataset, eval_dataset = train_test_split(dataset, test_size=0.1)

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
def encode_data(data, tokenizer, max_length=512):
    inputs = [tokenizer.encode_plus(i, max_length=max_length, truncation=True, padding="max_length", return_tensors="pt") for i in data['instruction']]
    labels = [tokenizer.encode(r, max_length=max_length, truncation=True, padding="max_length", return_tensors="pt") for r in data['response']]
    return inputs, labels

In [ ]:
from torch.utils.data import DataLoader, Dataset

class BankingDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.inputs, self.labels = encode_data(data, tokenizer, max_length)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx]['input_ids'].squeeze(),
            'attention_mask': self.inputs[idx]['attention_mask'].squeeze(),
            'labels': self.labels[idx].squeeze()
        }

train_data = BankingDataset(train_dataset, tokenizer)
eval_data = BankingDataset(eval_dataset, tokenizer)

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
eval_loader = DataLoader(eval_data, batch_size=8)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
)

# Fine-tuning işlemini başlatın
trainer.train()

In [ ]:
# Modeli değerlendirin
results = trainer.evaluate()
print(results)


In [ ]:
# Fine-tune edilmiş modeli kaydetme
model.save_pretrained("./retail-banking-chatbot-model")
tokenizer.save_pretrained("./retail-banking-chatbot-model")

